In [ ]:
def mal_eng_pos(mal,eng,mal_eng):
    tag = mal + " || " + eng
    if tag in mal_eng.keys():
        mal_eng[tag] += 1
    else:
        mal_eng[tag] = 1
    
    return mal_eng

def eng_mal_pos(mal,eng,eng_mal):
    tag = eng + " || " + mal
    if tag in eng_mal.keys():
        eng_mal[tag] += 1
    else:
        eng_mal[tag] = 1
    
    return eng_mal


In [ ]:
def pos_tag_diff(sentence, eng_mal, mal_eng):
    data = sentence.split()
    index = 0
    for index in range(len(data)-1):
        if not "</" in data[index]:
            continue
        mal_word = ""
        eng_word = ""
        pos_mal = ""
        pos_eng = ""
        if data[index].startswith("<malay>") and data[index].endswith("</malay>"):
            if not data[index+1].startswith("<english>"):
                continue
            mal_word = data[index][7:-8]
            if mal_word == "":
                index += 1
                mal_word = data[index]
            pos_mal = mal_word
            if data[index+1].startswith("<english>") and data[index+1].endswith("</english>"):
                eng_word = data[index+1][9:-10]
            else:
                eng_word = data[index+1][9:]
            if eng_word == "":
              index += 1
              eng_word = data[index]
            pos_eng = eng_word
            mal_eng = mal_eng_pos(pos_mal,pos_eng,mal_eng)

        elif data[index].endswith("</malay>"):
            if not data[index+1].startswith("<english>"):
                continue
            mal_word = data[index][:-8]
            if mal_word == "":
                index += 1
                mal_word = data[index]
            pos_mal = mal_word
            if data[index+1].startswith("<english>") and data[index+1].endswith("</english>"):
                eng_word = data[index+1][9:-10]
            else:
                eng_word = data[index+1][9:]
            if eng_word == "":
              index += 1
              eng_word = data[index]
            pos_eng = eng_word
            mal_eng = mal_eng_pos(pos_mal,pos_eng,mal_eng)

        elif data[index].startswith("<english>") and data[index].endswith("</english>"):
            if not data[index+1].startswith("<malay>"):
                continue
            eng_word = data[index][9:-10]
            if eng_word == "":
              index += 1
              eng_word = data[index]
            pos_eng = eng_word
            if data[index+1].startswith("<malay>") and data[index+1].endswith("</malay>"):
                mal_word = data[index+1][7:-8]
            else:
                mal_word = data[index+1][7:]
            if mal_word == "":
                index += 1
                mal_word = data[index]
            pos_mal = mal_word
            eng_mal = eng_mal_pos(pos_mal,pos_eng,eng_mal)

        elif data[index].endswith("</english>"):
            if not data[index+1].startswith("<malay>"):
                continue
            eng_word = data[index][:-10]
            if eng_word == "":
              index += 1
              eng_word = data[index]
            pos_eng = eng_word
            if data[index+1].startswith("<malay>") and data[index+1].endswith("</malay>"):
                mal_word = data[index+1][7:-8]
            else:
                mal_word = data[index+1][7:]
            if mal_word == "":
                index += 1
                mal_word = data[index]
            pos_mal = mal_word
            eng_mal = eng_mal_pos(pos_mal,pos_eng,eng_mal)
    return eng_mal, mal_eng

In [ ]:
import pandas as pd

filenametrain = "POS_CS_Lang_Data.txt"
with open(filenametrain) as f:
    data_train = f.readlines()

In [ ]:
eng_mal = dict()
mal_eng = dict()
for sentence in data_train:
    if "<english>" in sentence and "<malay>" in sentence:
        eng_mal, mal_eng = pos_tag_diff(sentence, eng_mal, mal_eng)

In [ ]:
def eng_tag(left,right,eng):
    resulting_model_eng = [left, right]
    pos1 = resulting_model_eng[0]
    pos2 = resulting_model_eng[1]
    tag = pos1 + " || " + pos2
    if tag in eng.keys():
        eng[tag] += 1
    else:
        eng[tag] = 1
    return eng

In [ ]:
def mal_tag(left,right,mal):
    resulting_model_mal = [left, right]
    pos1 = resulting_model_mal[0]
    try: 
      pos2 = resulting_model_mal[1]
    except:
      pos2 = pos1
    tag = pos1 + " || " + pos2
    if tag in mal.keys():
        mal[tag] += 1
    else:
        mal[tag] = 1
    return mal

In [ ]:
def pos_tag_same(sentence, eng, mal):
    data = sentence.split()
    index = 0
    eng_flag = False
    mal_flag = False
    for index in range(len(data)-1):
        left_word = data[index]
        right_word = ""
        if (eng_flag or left_word.startswith("<english>")) and not left_word.endswith("</english>"):
            if left_word.startswith("<english>"):
                left_word = left_word[9:]
            if left_word == "":
                index += 1
                left_word = data[index]
            if left_word.endswith("</english>"):
                continue
            eng_flag = True
            right_word = data[index+1]
            if right_word == "":
                index += 1
                right_word = data[index]
            if right_word.endswith("</english>"):
                right_word = right_word[:-10]
                eng_flag = False
            eng = eng_tag(left_word,right_word,eng)
        if (mal_flag or left_word.startswith("<malay>")) and not left_word.endswith("</malay>"):
            if left_word.startswith("<malay>"):
                left_word = left_word[7:]
            if left_word == "":
                index += 1
                left_word = data[index]
            if left_word.endswith("</malay>"):
                continue
            mal_flag = True
            right_word = data[index+1]
            if right_word == "":
                index += 1
                right_word = data[index]
            if right_word.endswith("</malay>"):
                right_word = right_word[:-8]
                mal_flag = False
            mal = mal_tag(left_word,right_word,mal)
    return eng,mal

In [ ]:
eng_eng = dict()
mal_mal = dict()
for sentence in data_train:
    if "<english>" in sentence or "<malay>" in sentence:
        eng_eng, mal_mal = pos_tag_same(sentence, eng_eng, mal_mal)

In [ ]:
column = ["POS || POS", "English to Malay", "English to English", "English to Malay Conversion Ratio", "Malay to English", "Malay to Malay", "Malay to English Conversion Ratio"]
res = pd.DataFrame(columns = column)

In [ ]:
pos_tag_list = ["NOUN", "PRON", "VERB", "ADVB", "ADJC", "ADPN", "CONJ", "DETR", "PCLE", "NMBR", "SYMB", "OTHR"]
for pos1 in pos_tag_list:
    for pos2 in pos_tag_list:
        res.loc[len(res.index)] = [pos1 + " || " + pos2, 0, 0, 0, 0, 0, 0]

for i in range(len(res.index)):
    if res.at[i,"POS || POS"] in mal_eng:
        res.at[i,"Malay to English"] = mal_eng[res.at[i,"POS || POS"]]
    else:
        res.at[i,"Malay to English"] = 0
    
    if res.at[i,"POS || POS"] in eng_mal:
        res.at[i,"English to Malay"] = eng_mal[res.at[i,"POS || POS"]]
    else:
        res.at[i,"English to Malay"] = 0
    
    if res.at[i,"POS || POS"] in eng_eng:
        res.at[i,"English to English"] = eng_eng[res.at[i,"POS || POS"]]
    else:
        res.at[i,"English to English"] = 0
    
    if res.at[i,"POS || POS"] in mal_mal:
        res.at[i,"Malay to Malay"] = mal_mal[res.at[i,"POS || POS"]] 
    else:
        res.at[i,"Malay to Malay"] = 0

In [ ]:
for i in range(len(res.index)):
    if res.at[i,"English to English"] == 0:
        res.at[i,"English to Malay Conversion Ratio"] = 0
    else:
        res.at[i,"English to Malay Conversion Ratio"] = round(res.at[i,"English to Malay"]*100/(res.at[i,"English to Malay"] + res.at[i,"English to English"]))
    
    if res.at[i,"Malay to Malay"] == 0:
        res.at[i,"Malay to English Conversion Ratio"] = 0
    else:
        res.at[i,"Malay to English Conversion Ratio"] = round(res.at[i,"Malay to English"]*100/(res.at[i,"Malay to English"] + res.at[i,"Malay to Malay"]))


In [ ]:
res.to_excel("Left_Right_POS_Results.xlsx", index = False)